Scraping pagina Web Colombia TIC

## Análisis de Reportes de Colombia TIC

Este notebook permite automatizar la descarga de reportes de Colombia TIC, extraer y procesar datos relevantes, 
y finalmente combinar la información en un solo DataFrame para análisis.

In [50]:
# librerias necesarias para el proyecto

import os
import re
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## Configuración Inicial

In [51]:
# Configuración de la URL base para la descarga y el uso de carpetas de almacenamiento.

BASE_URL = "https://colombiatic.mintic.gov.co/679/w3-channel.html"
download_base_path = "reporteColombiaTic"
combined_data_path = "reporteCombinado"

# ### Configuración de Selenium WebDriver

# Configurar el servicio de Chrome con la ruta completa
service = Service("C:/Users/Josvaldes/Documents/Maestria/Austral/trabajoGrado/trabajoGradoMCD/Codigo/chromedriver.exe")

# Opciones de Chrome (opcional, para correr en modo headless)
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

try:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(BASE_URL)
    print("Título de la página de prueba:", driver.title)
except Exception as e:
    print("Error al inicializar ChromeDriver:", e)
finally:
    driver.quit()

Título de la página de prueba: Inicio


## Funciones

In [52]:
# ### Función para crear subcarpetas

def create_directories(base_path, year, trimester):
    """
    Crea directorios para almacenar los archivos según año y trimestre.
    """
    path = os.path.join(base_path, f"Año_{year}", f"Trimestre_{trimester}")
    os.makedirs(path, exist_ok=True)
    return path

# ### Función para descargar el reporte

def download_report(url, save_path):
    """
    Descarga el archivo Excel desde la URL proporcionada y lo guarda en el path especificado.
    """
    response = requests.get(url)
    filename = os.path.join(save_path, "reporte_tic_trimestral.xlsx")
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"Reporte descargado en {filename}")
    return filename

# ## Descarga y Almacenamiento del Reporte

def archivo_trimestral_existente(nombre_archivo):
    return os.path.exists(nombre_archivo)

# Función principal para descargar el reporte
# Definir la ruta de la carpeta donde se guardará el archivo descargado
download_base_path = "reporteColombiaTic"

def fetch_report(url):
    """
    Navega en la página de Colombia TIC, encuentra y descarga el archivo Excel más reciente.
    """
    # Configurar el servicio de Chrome
    service = Service('C:/Users/Josvaldes/Documents/Maestria/Austral/trabajoGrado/trabajoGradoMCD/Codigo/chromedriver.exe')
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=service, options=options)
    
    driver.get(url)

    try:
        # Esperar y encontrar la sección "Destacados"
        destacados_header = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//h2[@class='ntg-titulo-caja' and text()='Destacados']"))
        )
        print("Se encontró la sección 'Destacados'.")

        # Buscar el enlace al artículo en la sección "Destacados"
        destacados_section = driver.find_element(By.XPATH, "//div[@class='recuadro col-md-6 col-lg-4 text-center mb-4']")
        link_element = destacados_section.find_element(By.TAG_NAME, "a")
        report_url = link_element.get_attribute('href')

        print("URL del artículo:", report_url)

        # Extraer el trimestre y año del reporte a partir del texto del enlace
        trimestre_texto = link_element.text
        trimestre = re.search(r"(primer|segundo|tercer|cuarto) trimestre de (\d{4})", trimestre_texto, re.IGNORECASE)
        if trimestre:
            trimestre_nombre = trimestre.group(1)
            año = trimestre.group(2)
            nombre_archivo = f"reporte_tic_{trimestre_nombre}_trimestre_{año}.xlsx"
        else:
            print("No se pudo determinar el trimestre del reporte.")
            nombre_archivo = "reporte_tic_trimestral.xlsx"

        # Crear la carpeta de descarga si no existe
        os.makedirs(download_base_path, exist_ok=True)
        
        # Ruta completa donde se guardará el archivo
        archivo_path = os.path.join(download_base_path, nombre_archivo)
        
        # Verificar si el archivo del trimestre ya existe
        if os.path.exists(archivo_path):
            print(f"El archivo '{archivo_path}' ya existe. No se realizará la descarga.")
            return archivo_path
        else:
            # Navegar a la página del artículo
            driver.get(report_url)

            # Esperar a que el `<span>` que contiene el enlace al archivo Excel esté presente
            excel_span = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'bajardoc') and contains(@class, 'binary-archivo_xls') and contains(@class, 'format-xlsx')]"))
            )

            # Obtener el enlace al archivo Excel
            excel_link = excel_span.find_element(By.TAG_NAME, "a")
            excel_url = excel_link.get_attribute('href')
            print("URL del archivo Excel:", excel_url)

            # Descargar el archivo Excel usando requests
            excel_response = requests.get(excel_url)

            # Verificar el código de estado antes de guardar
            if excel_response.status_code == 200:
                with open(archivo_path, 'wb') as file:
                    file.write(excel_response.content)
                print(f"Reporte descargado exitosamente como {archivo_path}")
                return archivo_path
            else:
                print(f"Error al descargar el archivo. Código de estado: {excel_response.status_code}")
                return None

    except Exception as e:
        print("No se encontró la sección 'Destacados' o hubo un error:", str(e))
        return None

    finally:
        # Cerrar el navegador
        driver.quit()


# Función para renombrar columnas duplicadas con sufijos secuenciales
def rename_duplicates(columns):
    """
    Renombra columnas duplicadas añadiendo un sufijo '_dupN' para evitar conflictos de nombres.
    """
    counts = {}
    new_columns = []
    for col in columns:
        if col in counts:
            counts[col] += 1
            new_columns.append(f"{col}_dup{counts[col]}")
        else:
            counts[col] = 0
            new_columns.append(col)
    return new_columns

def save_large_df_to_excel(df, file_path, chunk_size=1000000):
    """
    Guarda un DataFrame grande en un archivo Excel, dividiéndolo en múltiples hojas si es necesario.
    Args:
        df (DataFrame): DataFrame a guardar.
        file_path (str): Ruta del archivo Excel de salida.
        chunk_size (int): Número máximo de filas por hoja.
    """
    with pd.ExcelWriter(file_path) as writer:
        for i in range(0, len(df), chunk_size):
            df_chunk = df.iloc[i:i + chunk_size]
            sheet_name = f"Sheet_{i // chunk_size + 1}"
            df_chunk.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo combinado final guardado en {file_path}")

def save_large_df_to_csv(df, base_path, chunk_size=1_000_000):
    """
    Guarda un DataFrame grande en múltiples archivos CSV en una subcarpeta especificada.
    
    Parámetros:
    - df: DataFrame a guardar.
    - base_path: Ruta base para guardar los archivos CSV.
    - chunk_size: Tamaño del lote en número de filas para dividir el archivo CSV si es grande.
    """
    # Asegurarse de que la carpeta base exista
    os.makedirs(base_path, exist_ok=True)

    # Dividir y guardar el DataFrame en archivos CSV en chunks
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i + chunk_size]
        file_path = os.path.join(base_path, f"combined_data_part_{i // chunk_size + 1}.csv")
        chunk.to_csv(file_path, index=False)
        print(f"Guardado el archivo {file_path}")

# ## Procesamiento del Reporte

def process_report2(file_path):
    """
    Procesa cada hoja del archivo descargado, aplica limpieza y estandarización de columnas.
    """
    # Definir las columnas esperadas para cada hoja
    column_structure = {
        '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
        '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
        '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
        '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
        '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
        '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
        '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
        '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
        '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
        '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
        '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
        '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
    }

    all_sheets = pd.read_excel(file_path, sheet_name=None)
    processed_sheets = {}
    
    # Función para renombrar columnas duplicadas
    def rename_duplicates(columns):
        counts = {}
        new_columns = []
        for col in columns:
            if col in counts:
                counts[col] += 1
                new_columns.append(f"{col}_dup{counts[col]}")
            else:
                counts[col] = 0
                new_columns.append(col)
        return new_columns

    # Limpiar cada hoja y ajustar las columnas
    for sheet_name, df in all_sheets.items():
        if sheet_name not in column_structure:
            print(f"Omitiendo la hoja '{sheet_name}' (sin estructura conocida)")
            continue

        # Limpiar encabezados y resetear índices
        df = df.rename(columns=lambda x: x.strip()).dropna(how='all')
        
        # Renombrar columnas duplicadas
        df.columns = rename_duplicates(df.columns)

        # Validar si el número de columnas coincide
        if len(df.columns) == len(column_structure[sheet_name]):
            df.columns = column_structure[sheet_name]
            processed_sheets[sheet_name] = df
        else:
            print(f"Omitiendo la hoja '{sheet_name}' debido a la longitud de columnas no coincidente.")
    
    # Guardar un archivo temporal para depuración
    temp_output_path = os.path.join("reporteCombinado", "temp_combined_debug.xlsx")
    with pd.ExcelWriter(temp_output_path) as writer:
        for sheet_name, df in processed_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo temporal guardado en: {temp_output_path}")

    # Combinar todas las hojas en un solo DataFrame
    combined_df = processed_sheets.pop('1')
    for sheet_name, df in processed_sheets.items():
        # Verificar y renombrar columnas duplicadas que podrían causar problemas en el merge
        for col in df.columns:
            if col in combined_df.columns and col != 'AÑO' and col != 'TRIMESTRE':
                df = df.rename(columns={col: f"{col}_{sheet_name}"})

        # Obtener columnas comunes
        common_columns = combined_df.columns.intersection(df.columns).tolist()
        combined_df = pd.merge(combined_df, df, on=common_columns, how='outer', suffixes=('', f'_{sheet_name}'))

    return combined_df


def process_report3(file_path, temp_folder="reporteCombinado/temp_lotes", min_column_count=5, batch_size=10):
    """
    Procesa cada hoja de un archivo Excel, la guarda en archivos temporales por lotes y luego combina los resultados.
    Args:
        file_path (str): Ruta del archivo Excel a procesar.
        temp_folder (str): Carpeta para guardar los archivos procesados temporalmente.
        min_column_count (int): Número mínimo de columnas para procesar una hoja.
        batch_size (int): Número de archivos en cada lote para la combinación por lotes.
    Returns:
        DataFrame: DataFrame combinado de todas las hojas procesadas.
    """
    # Crear la carpeta temporal para guardar archivos procesados
    os.makedirs(temp_folder, exist_ok=True)
    
    # Cargar el archivo Excel
    excel_file = pd.ExcelFile(file_path)
    
    # Paso 1: Procesa cada hoja y guarda el resultado individualmente
    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja
            df = excel_file.parse(sheet_name)
            
            # Verificar la estructura de la hoja antes de procesar
            if len(df.columns) < min_column_count:
                print(f"Omitiendo la hoja '{sheet_name}' debido a longitud de columnas insuficiente.")
                continue
            
            # Limpiar los nombres de las columnas y renombrar duplicados
            df.columns = [col.strip() for col in df.columns]  # Quitar espacios en blanco
            df.columns = rename_duplicates(df.columns)
            
            # Guardar cada hoja procesada en un archivo temporal CSV
            temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
            df.to_csv(temp_file_path, index=False)
            print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")

        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")

    # Paso 2: Combina los archivos temporales en lotes
    # Lista de archivos temporales procesados
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]

    # DataFrame combinado final
    combined_df = pd.DataFrame()

    # Combinación por lotes
    for i in range(0, len(temp_files), batch_size):
        batch_files = temp_files[i:i + batch_size]
        
        # Leer y combinar el lote actual
        batch_df = pd.concat([pd.read_csv(f) for f in batch_files], axis=0, ignore_index=True)
        
        # Combina el lote con el DataFrame final
        if not combined_df.empty:
            common_columns = combined_df.columns.intersection(batch_df.columns).tolist()
            combined_df = pd.merge(combined_df, batch_df, on=common_columns, how='outer', suffixes=('', '_dup'))
        else:
            combined_df = batch_df.copy()  # Inicializar con el primer lote
        
        print(f"Lote {i // batch_size + 1} combinado exitosamente.")

    return combined_df

# ## Guardar el DataFrame Combinado

def save_combined_data(df, path):
    """
    Guarda el DataFrame combinado en un archivo Excel en la carpeta 'reporteCombinado'.
    """
    output_path = os.path.join(path, "data_reporte_combined_final.xlsx")
    os.makedirs(path, exist_ok=True)
    df.to_excel(output_path, index=False)
    print(f"DataFrame combinado guardado en: {output_path}")



# ## Análisis y Validación de Datos

# ### Función para verificar valores únicos y posibles inconsistencias

def analyze_columns(df):
    """
    Muestra valores únicos y verifica consistencias en las columnas clave.
    """
    columnas_clave = ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET']
    
    # Mostrar valores únicos en columnas clave
    for col in columnas_clave:
        if col in df.columns:
            unique_values = df[col].unique()
            print(f"\nValores únicos en '{col}':", unique_values[:10])  # Muestra los primeros 10 valores únicos

# ### Función para identificar y manejar datos nulos

def handle_missing_values(df):
    """
    Identifica y maneja valores nulos en el DataFrame, aplicando reemplazos si es necesario.
    """
    missing_info = df.isnull().sum()
    print("\nValores nulos en cada columna:\n", missing_info)
    
    # Puedes aplicar reemplazos según lo necesario, por ejemplo, reemplazar NaN en algunas columnas
    df['No. ACCESOS FIJOS A INTERNET'] = df['No. ACCESOS FIJOS A INTERNET'].fillna(0)
    # Más reemplazos pueden añadirse aquí según los análisis
    return df

# ### Función para generar estadísticas descriptivas

def generate_statistics(df):
    """
    Genera estadísticas descriptivas para las columnas numéricas del DataFrame.
    """
    numeric_cols = df.select_dtypes(include=['number']).columns
    print("\nEstadísticas descriptivas de columnas numéricas:\n", df[numeric_cols].describe())

# ### Guardar versión final del DataFrame para análisis

def save_final_data(df):
    """
    Guarda el DataFrame limpio y validado en un archivo Excel en la subcarpeta /reporteCombinado para análisis final.
    """
    final_output_path = os.path.join(combined_data_path, "data_reporte_combined_validated.xlsx")
    df.to_excel(final_output_path, index=False)
    print(f"DataFrame final validado guardado en: {final_output_path}")


# Carpeta para los archivos procesados temporalmente
temp_folder = "reporteCombinado/temp_lotes"
os.makedirs(temp_folder, exist_ok=True)

# Mínimo número de columnas esperadas para cada hoja (ajustar según tu estructura)
cierto_numero_de_columnas_minimo = 4

# Estructura esperada de columnas por hoja (añadir según tu estructura)
column_structure = {
    '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
    '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
    '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
    '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
    '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
    '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
    '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
    '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
    '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
    '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
    '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
    '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
    '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
    '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
    '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
    '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
}

def limpiar_hoja(df, columnas_esperadas):
    """
    Limpia una hoja eliminando encabezados adicionales y filas vacías, 
    y aplica la estructura de columnas esperada.
    """
    df = df.dropna(how='all')  # Elimina filas completamente vacías
    df.columns = df.columns.str.strip()  # Limpia espacios en nombres de columnas

    # Encuentra la primera fila válida de datos
    valid_row_index = df[df.iloc[:, 0].astype(str).str.match(r'^\d{4}$')].index.min()
    if valid_row_index is not None:
        df = df.iloc[valid_row_index:]
        df.columns = columnas_esperadas  # Aplica las columnas esperadas
        df = df.reset_index(drop=True)  # Reinicia el índice
    else:
        print(f"No se encontraron filas válidas en la hoja con columnas {columnas_esperadas}.")
    
    return df

def process_report(file_path):
    """
    Procesa cada hoja del archivo descargado, guarda cada una como CSV temporal,
    y luego combina todos los CSV temporales en un archivo final.
    """
    # Cargar cada hoja individualmente para evitar errores de archivos ZIP
    for sheet_name in pd.ExcelFile(file_path).sheet_names:
        try:
            # Intenta leer la hoja
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            # Verifica si el número de columnas es suficiente
            if len(df.columns) < cierto_numero_de_columnas_minimo:
                print(f"Omitiendo la hoja '{sheet_name}' debido a longitud de columnas insuficiente.")
                continue

            # Limpiar la hoja si tiene columnas esperadas
            if sheet_name in column_structure:
                columnas_esperadas = column_structure[sheet_name]
                if len(df.columns) == len(columnas_esperadas):
                    df = limpiar_hoja(df, columnas_esperadas)
                else:
                    print(f"Omitiendo la hoja '{sheet_name}' debido a la longitud de columnas no coincidente.")
                    continue

            # Guarda la hoja procesada en un archivo CSV temporal
            temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
            df.to_csv(temp_file_path, index=False)
            print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")

        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")

    # Combina los archivos CSV temporales por lotes
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]
    batch_size = 10  # Tamaño del lote de combinación para manejar memoria
    combined_df = pd.DataFrame()

    for i in range(0, len(temp_files), batch_size):
        batch_files = temp_files[i:i + batch_size]
        batch_df = pd.concat([pd.read_csv(f, low_memory=False) for f in batch_files], axis=0, ignore_index=True)
        
        # Realiza la combinación por lotes
        common_columns = combined_df.columns.intersection(batch_df.columns).tolist() if not combined_df.empty else batch_df.columns
        combined_df = pd.merge(combined_df, batch_df, on=common_columns, how='outer', suffixes=('', '_dup'))
        
        print(f"Lote {i // batch_size + 1} combinado exitosamente.")

    # Guarda el archivo final combinado
    combined_data_path = "reporteCombinado/combined_data_final.csv"
    combined_df.to_csv(combined_data_path, index=False)
    print(f"Archivo combinado final guardado en {combined_data_path}")




Inicio del programa

In [53]:
# ## Ejecución del Script

# Descargar y procesar el reporte
file_path = fetch_report(BASE_URL)  # Descarga y guarda el archivo en subcarpeta /reporteColombiaTic


Se encontró la sección 'Destacados'.
URL del artículo: https://colombiatic.mintic.gov.co/679/w3-article-397520.html
El archivo 'reporteColombiaTic\reporte_tic_segundo_trimestre_2024.xlsx' ya existe. No se realizará la descarga.


In [54]:
file_path 

'reporteColombiaTic\\reporte_tic_segundo_trimestre_2024.xlsx'

In [55]:
combined_df = process_report(file_path)  # Procesa y combina las hojas del reporte

# Guardar el DataFrame combinado en formato CSV
#combined_data_path_csv = "reporteCombinado/combined_data_final.csv"
#combined_df.to_csv(combined_data_path_csv, index=False)
#print(f"Archivo combinado final guardado en {combined_data_path_csv}")

#combined_data_path = "reporteCombinado"
#save_large_df_to_csv(combined_df, combined_data_path)


Omitiendo la hoja 'PORTADA' debido a longitud de columnas insuficiente.
Hoja 'CONTENIDO' procesada y guardada en reporteCombinado/temp_lotes\CONTENIDO_processed.csv
Hoja '1' procesada y guardada en reporteCombinado/temp_lotes\1_processed.csv
Hoja '2' procesada y guardada en reporteCombinado/temp_lotes\2_processed.csv
Hoja '3' procesada y guardada en reporteCombinado/temp_lotes\3_processed.csv
Hoja '4,1' procesada y guardada en reporteCombinado/temp_lotes\4,1_processed.csv
Hoja '4,2' procesada y guardada en reporteCombinado/temp_lotes\4,2_processed.csv
Hoja '4,3' procesada y guardada en reporteCombinado/temp_lotes\4,3_processed.csv
Hoja '4,4' procesada y guardada en reporteCombinado/temp_lotes\4,4_processed.csv
Hoja '5' procesada y guardada en reporteCombinado/temp_lotes\5_processed.csv
Hoja '6' procesada y guardada en reporteCombinado/temp_lotes\6_processed.csv
Hoja '7' procesada y guardada en reporteCombinado/temp_lotes\7_processed.csv
Hoja '8' procesada y guardada en reporteCombinado

KeyError: Index(['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS',
       'TRÁFICO (MB)', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO',
       'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS',
       'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO',
       'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES',
       'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'CÓDIGO DANE DEPARTAMENTO',
       'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO',
       'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS',
       'INGRESOS TELEFONIA FIJA', 'No. ACCESOS FIJOS A INTERNET',
       'CÓDIGO DANE', 'POBLACIÓN DANE', 'PENETRACIÓN'],
      dtype='object')

In [ ]:
StopIteration

StopIteration

In [ ]:
# Guardar el DataFrame combinado en un archivo final
combined_data_path = "reporteCombinado/combined_data_final.xlsx"
combined_df.to_excel(combined_data_path, index=False)
print(f"Archivo combinado final guardado en {combined_data_path}")

ValueError: This sheet is too large! Your sheet size is: 2246815, 16 Max sheet size is: 1048576, 16384

In [ ]:
# Guardar el DataFrame combinado en un archivo Excel en la subcarpeta /reporteCombinado
save_combined_data(combined_df, combined_data_path)

In [ ]:
# ## Ejecución de Validación y Análisis

# Realizar análisis de columnas clave
analyze_columns(combined_df)

# Manejar valores nulos
combined_df = handle_missing_values(combined_df)

# Generar estadísticas descriptivas
generate_statistics(combined_df)

# Guardar la versión final del DataFrame validado
save_final_data(combined_df)